<a href="https://colab.research.google.com/github/alammobaDar/Data_Scraping/blob/main/Activity1_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install pandas

In [ ]:
!pip install requests


# Scraping two website

In [57]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

# URLs to be scraped
urls = [
    'https://www.sideshow.com/blog/the-top-10-captain-america-quotes-in-the-mcu/',
    'https://everydaypower.com/captain-america-quotes/'
]

def clean_text(text):
    # Define unwanted phrases and patterns
    unwanted_patterns = [
        r'You will also enjoy our article on[^.]*\.',
        r'Want \$\d+ off\?Sign up for the newsletter',
        r'Check out our collection ofCaptain Price quotesif you enjoy this article.',
        r'Don’t forget to also check out thesePennywise quotesfrom the scariest clown of all.'
    ]
    for pattern in unwanted_patterns:
        text = re.sub(pattern, '', text)
    return text.strip()

def extract_text_from_url(url):
    # Send a GET request to the page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = bs(response.text, 'html.parser')

        quotes = []

        h2_tags = soup.find_all('h2')

        if url == 'https://everydaypower.com/captain-america-quotes/':
            specific_h2 = soup.find('h2', id='h-captain-america-quotes-from-captain-america-the-first-avenger')
            another_specific_h2 = soup.find('h2', id='h-captain-america-quotes-from-captain-america-the-winter-soldier')
            if specific_h2:
                section_paragraph = []
                for sibling in specific_h2.find_next_siblings():
                    if sibling.name == "h2":
                        break
                    elif sibling.name == "p":
                        section_paragraph.append(sibling.get_text(strip=True))
                section_text = ' '.join(section_paragraph)
                cleaned_text = clean_text(section_text)
                quotes.extend(cleaned_text.split('– Steve Rogers (Captain America)'))  # Split quotes based on period (or other delimiter)
            if another_specific_h2:
                section_paragraph = []
                for sibling in another_specific_h2.find_next_siblings():
                    if sibling.name == "h2":
                        break
                    elif sibling.name == "p":
                        section_paragraph.append(sibling.get_text(strip=True))
                section_text = ' '.join(section_paragraph)
                cleaned_text = clean_text(section_text)
                quotes.extend(cleaned_text.split('– Steve Rogers (Captain America)'))
        else:
            for h2 in h2_tags:
                h2_text = h2.get_text(strip=True)
                cleaned_text = clean_text(h2_text)

                quotes.extend(cleaned_text.split('\n'))  # Split quotes based on period (or other delimiter)

        # Return extracted text
        return [{'URL': url, 'Quotes': quote.strip()} for quote in quotes if quote.strip()]
    else:
        print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
        return []

# Extract information from both URLs and flatten the list
data = [quote for url in urls for quote in extract_text_from_url(url) if extract_text_from_url(url) is not None]

# Create DataFrame for structured information
df_info_01 = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df_info_01.to_csv('captain_america_quotes.csv', index=False)

# Print the DataFrame


In [39]:
df_info_01

,URL,Quote
0,https://www.sideshow.com/blog/the-top-10-capta...,“I don’t like bullies; I don’t care where they...
1,https://www.sideshow.com/blog/the-top-10-capta...,“On your left.”
2,https://www.sideshow.com/blog/the-top-10-capta...,“I’m just a kid from Brooklyn.”
3,https://www.sideshow.com/blog/the-top-10-capta...,“Language!”
4,https://www.sideshow.com/blog/the-top-10-capta...,“ThatisAmerica’s ass.”
5,https://www.sideshow.com/blog/the-top-10-capta...,“I can do this all day.”
6,https://www.sideshow.com/blog/the-top-10-capta...,“I’m with you ’til the end of the line.”
7,https://www.sideshow.com/blog/the-top-10-capta...,"“I’m not looking for forgiveness, and I’m way ..."
8,https://www.sideshow.com/blog/the-top-10-capta...,“Avengers! Assemble.”
9,https://www.sideshow.com/blog/the-top-10-capta...,"“Compromise where you can. Where you can’t, do..."




#Trying another website



In [52]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time
import re

# URL to be scraped
url = 'https://screenrant.com/captain-america-inspirational-quotes-marvel-comics/'

#remove unnecessaty text

def clean_text(text):
  unwanted_patterns = [
      r'Related',
      r'Trending Now'
  ]

  for pattern in unwanted_patterns:
    text = re.sub(pattern, '', text)
  return text.strip()

def extract_text_from_url(url):
    # Set a user-agent header
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the page
    try:
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Create BeautifulSoup object; parse with 'html.parser'
            soup = bs(response.text, 'html.parser')

            quotes = []
            h3_tags = soup.find_all('h3')

            for h3 in h3_tags:
                h3_text = h3.get_text(strip=True)
                section_paragraph = []
                section_paragraph.append(h3_text)
                section_text = '\n'.join(section_paragraph)
                cleaned_text = clean_text(section_text)
                quotes.extend(cleaned_text.split('\n'))
            return [{'URL': url, 'Quotes': quote.strip()} for quote in quotes if quote.strip()]
    except ConnectionError as e:
        print(f"Connection error occurred: {e}")
        return None
    finally:
        # Wait for a random time between 1 and 5 seconds
        time.sleep(random.uniform(1, 5))



# Extract information from the URL
data = extract_text_from_url(url)
print(data)
# Create DataFrame for structured information
if data:
    # Create DataFrame with URL as the first column
    df_info_02 = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df_info_02.to_csv('captain_america_screenrant_quotes.csv', index=False)
    # Print the DataFrame
    print(df_info_02)
else:
    print("No data to save.")


[{'URL': 'https://screenrant.com/captain-america-inspirational-quotes-marvel-comics/', 'Quotes': '"The Avengers should have done more to help mutants."'}, {'URL': 'https://screenrant.com/captain-america-inspirational-quotes-marvel-comics/', 'Quotes': '"If Captain America can\'t stand for the\xa0dream\xa0on the very\xa0worst\xa0day -when\xa0America\xa0is\xa0fallen - then there\xa0is\xa0no Captain America."'}, {'URL': 'https://screenrant.com/captain-america-inspirational-quotes-marvel-comics/', 'Quotes': '"As long as freedom may be threatened - Captain America must follow his destiny - wherever it may lead!"'}, {'URL': 'https://screenrant.com/captain-america-inspirational-quotes-marvel-comics/', 'Quotes': '"There is nothing sacred about the status quo... and there never will be!"'}, {'URL': 'https://screenrant.com/captain-america-inspirational-quotes-marvel-comics/', 'Quotes': '"Fight the Falcon, and you fight me too!"'}, {'URL': 'https://screenrant.com/captain-america-inspirational-quot

In [53]:
df_info_02

,URL,Quotes
0,https://screenrant.com/captain-america-inspira...,"""The Avengers should have done more to help mu..."
1,https://screenrant.com/captain-america-inspira...,"""If Captain America can't stand for the dream ..."
2,https://screenrant.com/captain-america-inspira...,"""As long as freedom may be threatened - Captai..."
3,https://screenrant.com/captain-america-inspira...,"""There is nothing sacred about the status quo...."
4,https://screenrant.com/captain-america-inspira...,"""Fight the Falcon, and you fight me too!"""
5,https://screenrant.com/captain-america-inspira...,"""What kind of world will this be without a Cap..."
6,https://screenrant.com/captain-america-inspira...,"""It is the dream... the hope... that makes the..."
7,https://screenrant.com/captain-america-inspira...,"""As long as one man stands against you, you'll..."
8,https://screenrant.com/captain-america-inspira...,"""I represent the American Dream."""
9,https://screenrant.com/captain-america-inspira...,"""No, you move."""


# Combining the Two Dataframe

In [60]:
import pandas as pd

# Load the CSV files into DataFrames
df_info_01 = pd.read_csv('captain_america_quotes.csv')
df_info_02 = pd.read_csv('captain_america_screenrant_quotes.csv')

# Concatenate the DataFrames
df_combined = pd.concat([df_info_01, df_info_02], ignore_index=True)

# Save the combined DataFrame to a new CSV file
df_combined.to_csv('combined_quotes.txt', index=False)

#Print combined DataFrame
print(df_combined)

                                                  URL  \
0   https://www.sideshow.com/blog/the-top-10-capta...   
1   https://www.sideshow.com/blog/the-top-10-capta...   
2   https://www.sideshow.com/blog/the-top-10-capta...   
3   https://www.sideshow.com/blog/the-top-10-capta...   
4   https://www.sideshow.com/blog/the-top-10-capta...   
5   https://www.sideshow.com/blog/the-top-10-capta...   
6   https://www.sideshow.com/blog/the-top-10-capta...   
7   https://www.sideshow.com/blog/the-top-10-capta...   
8   https://www.sideshow.com/blog/the-top-10-capta...   
9   https://www.sideshow.com/blog/the-top-10-capta...   
10  https://everydaypower.com/captain-america-quotes/   
11  https://everydaypower.com/captain-america-quotes/   
12  https://everydaypower.com/captain-america-quotes/   
13  https://everydaypower.com/captain-america-quotes/   
14  https://everydaypower.com/captain-america-quotes/   
15  https://everydaypower.com/captain-america-quotes/   
16  https://everydaypower.com/c

In [64]:
df_combined

,URL,Quotes
0,https://www.sideshow.com/blog/the-top-10-capta...,“I don’t like bullies; I don’t care where they...
1,https://www.sideshow.com/blog/the-top-10-capta...,“On your left.”
2,https://www.sideshow.com/blog/the-top-10-capta...,“I’m just a kid from Brooklyn.”
3,https://www.sideshow.com/blog/the-top-10-capta...,“Language!”
4,https://www.sideshow.com/blog/the-top-10-capta...,“ThatisAmerica’s ass.”
5,https://www.sideshow.com/blog/the-top-10-capta...,“I can do this all day.”
6,https://www.sideshow.com/blog/the-top-10-capta...,“I’m with you ’til the end of the line.”
7,https://www.sideshow.com/blog/the-top-10-capta...,"“I’m not looking for forgiveness, and I’m way ..."
8,https://www.sideshow.com/blog/the-top-10-capta...,“Avengers! Assemble.”
9,https://www.sideshow.com/blog/the-top-10-capta...,"“Compromise where you can. Where you can’t, do..."
